In [2]:
import numpy as np
import cvxpy as cp
import gurobipy as gp
import pandas as pd
#import mosek
import matplotlib.pyplot as plt
import phi_divergence as phi
from scipy.optimize import fsolve
import scipy.stats
from itertools import chain, combinations
import time
from datetime import datetime as dt
import scipy.stats
from dateutil.relativedelta import *

In [3]:
def generate_data_mohajerin2018(N, k):
    np.random.seed(1)
    sys_risk_mean = 0
    #sys_risk_stdev = math.sqrt(0.02)
    sys_risk_stdev = 0.02
    unsys_risk_mean = np.fromiter(((i * 0.03) for i in range(1,k+1)), float)
    #unsys_risk_stdev = np.fromiter(( math.sqrt(i * 0.025) for i in range(1,k+1)), float)
    unsys_risk_stdev = np.fromiter(( i * 0.025 for i in range(1,k+1)), float)
    data = np.empty([N,k])
    for n in range(0, N):
        sys_return = np.random.normal(sys_risk_mean, sys_risk_stdev)
        for i in range(0, k):
            unsys_return = np.random.normal(unsys_risk_mean[i], unsys_risk_stdev[i])
            data[n, i] = sys_return + unsys_return
    return data 


def generate_data_natarajan2008( N, k):    
    np.random.seed(1)
    gamma = np.fromiter((((1/2)*(1 + (i/(k+1)))) for i in range(1,k+1)), float)
    print(gamma)
    return_pos = np.fromiter(((np.sqrt((1-gamma[i])*gamma[i])/gamma[i]) for i in range(0,k)), float)
    return_neg = np.fromiter((-(np.sqrt((1-gamma[i])*gamma[i])/(1-gamma[i])) for i in range(0,k)), float)
    data = np.empty([N,k])
    for n in range(0, N):
        for i in range(0, k):
            prob = np.random.uniform()
            if prob <= gamma[i]:
                data[n, i] = return_pos[i]
            else:
                data[n, i] = return_neg[i]
    return data 

In [4]:
def make_slope_ic(x_d, y_d):
    slope = np.zeros(len(x_d)-1)
    for i in range(1,len(slope)+1):
        slope[i-1] = (y_d[i]-y_d[i-1])/(x_d[i]-x_d[i-1])
    i0 = np.where(np.diff(slope)>=0)[0][0]   #### slope[i0] is the last concave slope
    x_d1 = x_d[0:(i0)+2]   #### (x_d[0],....,x_d[i0+1]) are the x-points of all concave part
    y1 = y_d[0:(i0)+2]
    x_d2 = np.sort(1-x_d[(i0)+1: len(x_d)])  #####  (x_d[i0+1],...., x_d[len(x_d)-1]) are the x-points of convex parts, 1-(...) sorted are that of
                                                                                        #### the dual functions bar{h}(p)=1-h(1-p)
    y2 = np.sort(1-y_d[(i0)+1: len(y_d)])
    #print('slope',slope, 'x_d1', x_d1,'y1', y1)
    #print('x_d2',x_d2, 'y2', y2)
    slope1 = slope[0:(i0+1)]       #### select all the concave slopes: slope[0:(i0+1)]= (slope[0],....,slope[i0])
    slope2 = -np.sort(-slope[(i0+1):len(slope)])   #### The slopes of the dual-concave functions are 
                                                            ### the convex slopes (slope[i0+1],...,slope[len(slope)-1]) sorted backwards
    ic1 = np.zeros(len(slope1))
    ic2 = np.zeros(len(slope2))
    for i in range(len(slope1)):
        ic1[i] = y1[i] - slope1[i]*x_d1[i]

    for i in range(len(slope2)):
        ic2[i] = y2[i] - slope2[i]*x_d2[i]
    h_po = y1[len(y1)-1]
    h_hat = y2[len(y2)-1]
    return(slope1, ic1, slope2,ic2,h_po, h_hat,slope)
    
    

In [5]:
def S_distortion_opt(R,prob,slope1,ic1,slope2,ic2,h_po,h_hat):
    md = gp.Model("S_distortion")
    m = len(R)
    n_a = len(R[0])
    K1 = len(slope1)
    K2 = len(slope2)
    n_it = len(prob)
    beta = md.addVar(-gp.GRB.INFINITY)
    a = md.addVars(n_a)
    qb = md.addVars(m) 
    nu = md.addVars(K1)
    lbda = md.addVars(m, K1)
    t_ik = md.addVars(m, K2)
    md.addConstr(gp.quicksum(qb[i] for i in range(m)) == h_hat)
    md.addConstr(gp.quicksum(a[i] for i in range(n_a)) == 1)
    md.addConstrs((lbda[i,k] <= nu[k] for k in range(K1) for i in range(m)))
    md.addConstrs(gp.quicksum(-R[i,j]* a[j] for j in range(n_a))-1 -\
                      beta - gp.quicksum(lbda[i,k] for k in range(K1)) <= 0 for i in range(m))
    obj1 = beta*h_po + gp.quicksum(nu[k]*ic1[k] for k in range(K1)) + gp.quicksum(lbda[i,k]*prob[i]*slope1[k]\
                                            for k in range(K1) for i in range(m))
    md.addConstrs((qb[i] <= slope2[k]*prob[i] + t_ik[i,k] for k in range(K2) for i in range(m)))
    md.addConstrs(gp.quicksum(t_ik[i,k] for i in range(m)) <= ic2[k] for k in range(K2))
    md.setObjective(obj1 + gp.quicksum(-qb[i] * (gp.quicksum(a[j] * R[i, j] for j in range(n_a)) + 1) for i in range(m)), gp.GRB.MINIMIZE)
    #md.setObjective(obj1 + gp.quicksum(-qb[i]*a[j]*R[i,j] for j in range(n_a) for i in range(m))-h_hat, gp.GRB.MINIMIZE)
    md.params.NonConvex = 2
    md.params.OutputFlag = 0
    md.optimize()
    a_val = md.getAttr("X", a)
    return(md.objVal, np.array(a_val.values()), md.MIPGap*np.abs(md.objVal))

In [6]:
def S_dist_iterate(R,prob,slope1,ic1,slope2,ic2,h_po, h_hat):
    md = gp.Model("S_distortion")
    #md.params.MIPGap = 1e-7
    m = len(R)
    n_a = len(R[0])
    K1 = len(slope1)
    K2 = len(slope2)
    n_it = len(prob)
    u = md.addVars(n_it, lb = -gp.GRB.INFINITY)
    t_max = md.addVar(-gp.GRB.INFINITY)
    beta = [md.addVar(-gp.GRB.INFINITY) for i in range(n_it)]
    a = md.addVars(n_a)
    qb = [md.addVars(m) for i in range(n_it)] 
    nu = [md.addVars(K1) for i in range(n_it)]
    lbda = [md.addVars(m, K1) for i in range(n_it)]
    t_ik = [md.addVars(m, K2) for i in range(n_it)]
    for it in range(n_it):
        md.addConstr(gp.quicksum(qb[it][i] for i in range(m)) == h_hat)
        md.addConstr(gp.quicksum(a[i] for i in range(n_a)) == 1)
        md.addConstrs((lbda[it][i,k] <= nu[it][k] for k in range(K1) for i in range(m)))
        md.addConstrs(gp.quicksum(-R[i,j]* a[j] for j in range(n_a))-1 -\
                          beta[it] - gp.quicksum(lbda[it][i,k] for k in range(K1)) <= 0 for i in range(m))
        md.addConstr(beta[it]*h_po + gp.quicksum(nu[it][k]*ic1[k] for k in range(K1)) +\
                                         gp.quicksum(lbda[it][i,k]*prob[it][i]*slope1[k]\
                                                for k in range(K1) for i in range(m) if prob[it][i] >= 1e-11) <= u[it])
        for i in range(m):
            if prob[it][i]>= 1e-11:
                md.addConstrs((qb[it][i] <= slope2[k]*prob[it][i] + t_ik[it][i,k] for k in range(K2)))
            else:
                md.addConstrs((qb[it][i] <= t_ik[it][i,k] for k in range(K2)))
        md.addConstrs(gp.quicksum(t_ik[it][i,k] for i in range(m)) <= ic2[k] for k in range(K2))
        md.addConstr(u[it] + gp.quicksum(-qb[it][i]*(gp.quicksum(a[j]*R[i,j] for j in range(n_a))+1) for i in range(m)) <= t_max)
        #md.addConstr(u[it] + gp.quicksum(-qb[it][i]*a[j]*R[i,j] for j in range(n_a) for i in range(m))-h_hat <= t_max)
    md.setObjective(t_max, gp.GRB.MINIMIZE)
    md.params.NonConvex = 2
    md.params.OutputFlag = 0
    md.optimize()
    a_val = md.getAttr("X", a)
    u_val = md.getAttr("X", u)
    
    return(md.objVal, np.array(list(a_val.values())), md.MIPGap*np.abs(md.objVal))

In [7]:
def Robustness_S_dist(R,a,r,p,domain,value, quad):   #### Code for calculating robust evaluation of a single solution
                                                          ### quad = 1 for quadratic phi(t)=(t-1)^2, quad = 0 for phi(t)=|t-1|
    md = gp.Model("WCrob")
    m = len(p)
    N = len(domain)
    q = md.addVars(m)
    z = md.addVars(m)
    lambda_var = md.addVars(m, N)
    ind = np.argsort(-R.dot(a))
    x = -np.sort(-R.dot(a))
    dx = np.diff(-x)
    ind2 = np.argsort(ind)
    md.addConstr(gp.quicksum(q[i] for i in range(m)) == 1)
    if quad:
        md.addConstr(gp.quicksum(1/p[i]*(q[i] - p[i]) ** 2 for i in range(m)) <= r)
    else:
        s1 = md.addVars(m)                                        ##### These codes can be used when using phi(t)=|t-1|
        s2 = md.addVars(m, lb = -gp.GRB.INFINITY)
        md.addConstr(gp.quicksum(s1[i] for i in range(m)) <= r)
        md.addConstrs(s1[i] == gp.abs_(s2[i]) for i in range(m))
        md.addConstrs(s2[i] == p[i]-q[i] for i in range(m))
        #md.addConstr(gp.quicksum(gp.abs_(p[i] - q[i]) for i in range(m)) <= r, name="abs_sum_constraint")

        
    for i in range(m):
        sos2_indices = list(range(N))
        md.addSOS(gp.GRB.SOS_TYPE2, [lambda_var[i, j] for j in sos2_indices])
        md.addConstr(gp.quicksum(lambda_var[i,j] for j in sos2_indices) == 1)
        if i > 0:
            md.addConstr(gp.quicksum(lambda_var[i,j] * domain[j] for j in sos2_indices)== gp.quicksum(q[k] for k in range(i,m)))
            md.addConstr(z[i] == gp.quicksum(lambda_var[i,j] * value[j] for j in sos2_indices))
    dx2 = np.concatenate(([0],dx))
    objective_expr = -x[0] + gp.quicksum(z[i] * dx2[i] for i in range(1,m))
    md.setObjective(objective_expr, gp.GRB.MAXIMIZE)
    #md.params.OutputFlag = 0
    #md.params.BestBdStop = lb_tol
    #md.setParam('FeasibilityTol', 1e-9)

    md.optimize()
    obj_pw = md.objVal
    q_val = md.getAttr("X", q)
    q_val = np.array(list(q_val.values()))
    return(obj_pw-1, q_val[ind2])



In [8]:
def cutting_plane_Sdist(R,p,r,eps,slope1,ic1,slope2,ic2,h_po, h_hat, domain,value,it, quad):   #### Code solving robust portfolio
    gap = np.inf
    prob = [p]
    i = 0
    while gap > eps and i<= it:
        [obj,a_s,mip_gap] = S_dist_iterate(R,prob,slope1,ic1,slope2,ic2,h_po, h_hat)
        print('lb:',obj, 'lb_precision:',mip_gap)
        #lb_tol = obj + eps + 1.003
        #[wc, q_wc] = Robustness_S_dist_prestop(R, a_s, r, p, domain, value, quad, lb_tol)
        [wc, q_wc] = Robustness_S_dist(R,a_s,r,p,domain,value, quad)  #Robustness_S_dist2(R,a_s,r,p,slope1,ic1,slope2,ic2)
        gap = wc - obj
        print('ub:',wc, 'iter:', i, 'gap', gap)#, 'run time wc:',t21-t11)
        prob.append(q_wc)
        i = i + 1
    print(np.sum((q_wc-p)**2/p), np.sum(q_wc), np.min(q_wc))
    return('sol:',a_s, 'lb:',obj, 'ub:', obj + gap)

In [9]:
def Prelec(x, a):
    if x == 1:
        return 1
    else:
        return 1 - np.exp(-(-np.log(1 - x))**a)

def Prelec_dual(x,a):
    return 1-Prelec(1-x,a)

def Prelec_derivative(x,a):
    if x<1:
        return a*np.exp(-(-np.log(1 - x))**a)*(-np.log(1-x))**(a-1)*1/(1-x)

def Prelec_derivative_dual(x,a):
    return Prelec_derivative(1-x,a)

# Define the function to maximize (renamed to error_func)
def error_func(p, x_i, x_i1, a):
    # Compute the values of h(x_i) and h(x_i1) using Prelec
    h_xi = Prelec(x_i, a)
    h_xi1 = Prelec(x_i1, a)
    
    # Compute the function error_func(p)
    return -(Prelec(p, a) - (h_xi1 - h_xi) / (x_i1 - x_i) * (p - x_i) - h_xi)

def error_func_dual(p, x_i, x_i1, a):
    h_xi = Prelec_dual(x_i, a)
    h_xi1 = Prelec_dual(x_i1, a)
    
    # Compute the function error_func(p)
    return -(Prelec_dual(p, a) - (h_xi1 - h_xi) / (x_i1 - x_i) * (p - x_i) - h_xi)

def error_func_derivative(p, x_i, x_i1, a):
    h_xi = Prelec(x_i, a)
    h_xi1 = Prelec(x_i1, a)
    return -(Prelec_derivative(p,a) - (h_xi1 - h_xi) / (x_i1 - x_i))

def error_func_derivative_dual(p, x_i, x_i1, a):
    h_xi = Prelec_dual(x_i, a)
    h_xi1 = Prelec_dual(x_i1, a)
    return -(Prelec_derivative_dual(p,a) - (h_xi1 - h_xi) / (x_i1 - x_i))

def max_error(x_i, x_i1, a):
    # Check if x_i and x_i1 are within the allowed range
    if x_i < 0 or x_i1 > 1 - 1 / np.e or x_i >= x_i1:
        raise ValueError("Invalid input range: x_i must be in [0, 1-1/e], and x_i < x_i1.")
    
    # Bisection search for the root of the derivative
    left = x_i
    right = x_i1
    while True:  # Convergence threshold for the search
        mid = (left + right) / 2
        derivative_at_mid = error_func_derivative(mid, x_i, x_i1, a)
        if np.abs(right-left)<= 1e-16:
            print('reached machine precision, current derivative', derivative_at_mid)
            return(-error_func(mid, x_i, x_i1, a))
        if np.abs(derivative_at_mid)<= 1e-8:
            return -error_func(mid, x_i, x_i1, a)
        if derivative_at_mid > 0:
            right = mid  # Root is in the left half
        if derivative_at_mid < 0:
            left = mid  # Root is in the right half
    


def max_error_dual(x_i, x_i1, a):
    # Check if x_i and x_i1 are within the allowed range
    if x_i < 0 or x_i1 > 1 / np.e or x_i > x_i1:
        raise ValueError("Invalid input range: x_i must be in [0, 1/e], and x_i < x_i1.")
    
    
    # Bisection search for the root of the derivative
    left = x_i
    right = x_i1
    old_deri = -1e8
    while True:  # Convergence threshold for the search
        mid = (left + right) / 2
        derivative_at_mid = error_func_derivative_dual(mid, x_i, x_i1, a)
        if np.abs(right-left)<= 1e-16:
            print('reached machine precision for dual h, current derivative', derivative_at_mid)
            return(-error_func_dual(mid, x_i, x_i1, a))
        if np.abs(derivative_at_mid)<= 1e-7:
            return -error_func_dual(mid, x_i, x_i1, a)
        if derivative_at_mid > 0:
            right = mid  # Root is in the left half
        if derivative_at_mid < 0:
            left = mid  # Root is in the right half
        old_deri = derivative_at_mid
    


In [10]:
def find_x_sequence(epsilon, delta, a):
    x_values = [0]  # Start with x_0 = 0
    x_i = x_values[-1]  # Initialize x_i
    
    while True:
        # Initialize bounds for x_{i+1}
        left = x_i
        right = 1 - 1 / np.e  # Start with the largest possible x_{i+1}
        # Perform bisection search to find x_{i+1}
        if max_error(x_i, 1 - 1 / np.e, a) <= epsilon:
            x_values.append(1-1/np.e)
            break  # Terminate the process entirely
        while True:
            mid = (left + right) / 2  # Midpoint of the interval

            error_at_mid = max_error(x_i, mid, a)  # Compute the error at the midpoint
            # Check the termination condition
            if abs(error_at_mid - epsilon) < delta:
                x_i1 = mid
                break  # Exit the search for this interval
            
            # Adjust bounds based on the error
            if error_at_mid > epsilon:
                right = mid  # Increase x_{i+1}
            else:
                left = mid  # Decrease x_{i+1}
        
        # Append the new x_{i+1} to the sequence
        x_values.append(x_i1)
        x_i = x_i1
        
        # Check if the error for the interval [x_i, x_{i+1}] is acceptable
        
        
        # Move to the next interval
        
    
    return x_values


def find_x_sequence_dual(epsilon, delta, a):
    x_values = [0]  # Start with x_0 = 0
    x_i = x_values[-1]  # Initialize x_i
    
    while True:
        # Initialize bounds for x_{i+1}
        left = x_i
        right = 1 / np.e  # Start with the largest possible x_{i+1}
        # Perform bisection search to find x_{i+1}
        if max_error_dual(x_i, 1 / np.e, a) <= epsilon:
            x_values.append(1/np.e)
            break  # Terminate the process entirely
        while True:
            mid = (left + right) / 2  # Midpoint of the interval

            error_at_mid = max_error_dual(x_i, mid, a)  # Compute the error at the midpoint
            # Check the termination condition
            if abs(error_at_mid - epsilon) < delta:
                x_i1 = mid
                break  # Exit the search for this interval
            
            # Adjust bounds based on the error
            if error_at_mid > epsilon:
                right = mid  # Increase x_{i+1}
            else:
                left = mid  # Decrease x_{i+1}
        
        # Append the new x_{i+1} to the sequence
        x_values.append(x_i1)
        x_i = x_i1
        
        # Check if the error for the interval [x_i, x_{i+1}] is acceptable
        
        
        # Move to the next interval
        
    
    return x_values





In [12]:
a = 0.65
epsilon = 0.005
margin = 0.00001
x_sequence = find_x_sequence(epsilon, margin, a)
x_sequence_dual = find_x_sequence_dual(epsilon, margin, a)
x_d=np.concatenate((np.array(x_sequence),np.sort(1-np.array(x_sequence_dual)[0:len(x_sequence_dual)-1])))
#print(x_d)
#print(len(x_d))
y_d=np.zeros(len(x_d))
for i in range(len(y_d)):
    if i <= len(x_sequence)-1:
        y_d[i] = Prelec(x_d[i],a)
    else:
        y_d[i] = 1-Prelec_dual(1-x_d[i],a)
#print(y_d)
y_dl_2 = y_d.copy()
y_du_2 = y_d.copy()
x_dl_2 = x_d.copy()
x_du_2 = x_d.copy()
index_ub = np.argmax(y_d[0:len(x_sequence)]+epsilon >= Prelec(1-1/np.e,a))
if len(x_sequence)-1-index_ub == 0:
    x_point_ub = (Prelec(1-1/np.e,a) - y_d[index_ub-1]-epsilon) * (x_d[index_ub]-x_d[index_ub-1])/(y_d[index_ub]-y_d[index_ub-1]) + x_d[index_ub-1]
    x_du_2 = np.insert(x_du_2, index_ub, x_point_ub)
    y_du_2 = np.insert(y_du_2, index_ub, Prelec(1-1/np.e,a))
    for i in range(len(x_sequence)-1):
        y_du_2[i] = y_du_2[i]+epsilon
else:
    print('need smaller epsilon')

#print(x_du_2)
#print(y_du_2)

index_lb = np.argmin(y_d-epsilon <= Prelec(1-1/np.e,a))
if len(x_sequence)-index_lb == 0:
    x_point_lb = (Prelec(1-1/np.e,a) - y_d[index_lb-1]+epsilon) * (x_d[index_lb]-x_d[index_lb-1])/(y_d[index_lb]-y_d[index_lb-1]) + x_d[index_lb-1]
    x_dl_2 = np.insert(x_dl_2, index_lb, x_point_lb)
    y_dl_2 = np.insert(y_dl_2, index_lb, Prelec(1-1/np.e,a))
    for i in range(len(x_sequence)+1, len(x_dl_2)):
        y_dl_2[i] = y_dl_2[i]-epsilon
    
    #print(x_dl_2)
    #print(y_dl_2)

else:
    print('need smaller epsilon')








In [ ]:

#f_lin = interp1d(x_sequence, y_seq, kind='linear', fill_value="extrapolate")
x_grid = np.concatenate((np.arange(0,1,1/1000),np.array([1])))
y_grid_real_1 = np.zeros(len(x_grid))
y_grid_real_2 = np.zeros(len(x_grid))
y_grid_real_3 = np.zeros(len(x_grid))
for i in range(len(x_grid)):
    y_grid_real_1[i] = Prelec(x_grid[i],0.6)
    y_grid_real_2[i] = Prelec(x_grid[i],0.65)
    y_grid_real_3[i] = Prelec(x_grid[i],0.75)

In [ ]:
#plt.figure(figsize=(8, 6))
#plt.plot(x_du_1,y_du_1,linestyle = '--', color = 'black', linewidth = 1.5)
#plt.plot(x_dl_1,y_dl_1,linestyle = '--', color = 'green', linewidth = 1.5)
plt.plot(x_du_2,y_du_2,linestyle = '--', color = 'black', linewidth = 1.5)
plt.plot(x_dl_2,y_dl_2,linestyle = '--', color = 'green', linewidth = 1.5)
#plt.plot(x_du_3,y_du_3,linestyle = '--', color = 'black', linewidth = 1.5)
#plt.plot(x_dl_3,y_dl_3,linestyle = '--', color = 'green', linewidth = 1.5)
#plt.plot(x_grid,y_grid_real_1, color = 'blue', linewidth= 1, label = r'$\alpha = 0.6$')
plt.plot(x_grid,y_grid_real_2, color = 'red', linewidth= 1, label = r'$\alpha = 0.65$')
#plt.plot(x_grid,y_grid_real_3, color = 'orange', linewidth= 1, label = r'$\alpha = 0.75$')
plt.legend()
plt.savefig('Prelec_functons_ul.eps', format = 'eps')

In [13]:
np.random.seed(1)
R = generate_data_mohajerin2018(100, 5)
m = len(R)
p = np.zeros(m)+1/m
prob = p
r = 1/(m)*scipy.stats.chi2.ppf(0.95, m-1)  

In [ ]:
#################            test
#### The case of alpha = 0.65, lower bound, epsilon =0.005, 13 pieces
[slope1, ic1, slope2,ic2,h_po, h_hat,slope] = make_slope_ic(x_d, y_d)

ic2 = ic2 + epsilon ### This gives a lower bound
#### Solve nominal problem:
t1 = time.time()
res = S_distortion_opt(R,p,slope1,ic1,slope2,ic2,h_po,h_hat)   #### Running this code requires Gurobi License
t2 = time.time()
print('nominal solution')
print('obj:', res[0], 'sol:', res[1], 'precision:', res[2])
print('time:',t2-t1)

#### Solve robust problem
eps = 0.001                     ##### Code for running the robust problem for Prelec parameter with beta = 0.3
it = 30
quad = 1
t_b = time.time()
print('robust solution')
rob_results_lb = cutting_plane_Sdist(R,p,r,eps,slope1,ic1,slope2,ic2,h_po, h_hat, x_dl_2,y_dl_2,it,quad)
print(rob_results_lb)
t_e = time.time()
print(t_e-t_b)

In [ ]:
[slope1, ic1, slope2,ic2,h_po, h_hat,slope] = make_slope_ic(x_du_2, y_du_2)
a_lb = rob_results_lb[1]
#### Solve nominal problem:
t1 = time.time()
res = S_distortion_opt(R,p,slope1,ic1,slope2,ic2,h_po,h_hat)   #### Running this code requires Gurobi License
t2 = time.time()
print('nominal solution')
print('obj:', res[0], 'sol:', res[1], 'precision:', res[2])
print('time:',t2-t1)
print('upper bound robust solution')
tb = time.time()
wc_res_lb = Robustness_S_dist(R,a_lb,r,p,x_du_2,y_du_2, quad)
te = time.time()
print(wc_res_lb)

In [ ]:
print('gap',wc_res_lb[0]-rob_results_lb[3])
print('lb', rob_results_lb[3])
print('ub', wc_res_lb[0])
print('gap nominal', res_u[0]-res_l[0])
